In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [57]:
class Fluid:
    def __init__(self, size, step, scale, forceRange, iteration, end):
        self.size       = size
        self.step       = step
        self.scale      = scale
        self.forceRange = forceRange
        self.iteration  = iteration
        self.end        = end
        
        self.p = np.full((size, size), 0., dtype = float)
        self.u = np.full((size, size), 0., dtype = float)
        self.v = np.full((size, size), 0., dtype = float)
        self.r = np.full((size, size), 0., dtype = float)
        
        self.ptmp = np.full((size, size), 0., dtype = float)
        self.utmp = np.full((size, size), 0., dtype = float)
        self.vtmp = np.full((size, size), 0., dtype = float)
        self.rtmp = np.full((size, size), 0., dtype = float)
    
    def advect(self):
        for i in np.arange(self.size, dtype = int)[1 : -1]:
            for j in np.arange(self.size, dtype = int)[1 : -1]:
                xPos = i - self.u[j, i] * self.step / self.scale
                yPos = j - self.v[j, i] * self.step / self.scale
                x, y = int(xPos), int(yPos)

                dx = xPos - x
                dy = yPos - y

                a = dx * (self.u[y    , x + 1] - self.u[y    , x]) + self.u[y    , x]
                b = dx * (self.u[y + 1, x + 1] - self.u[y + 1, x]) + self.u[y + 1, x]
                self.utmp[j, i] = dy * (b - a) + a

                a = dx * (self.v[y    , x + 1] - self.v[y    , x]) + self.v[y    , x]
                b = dx * (self.v[y + 1, x + 1] - self.v[y + 1, x]) + self.v[y + 1, x]
                self.vtmp[j, i] = dy * (b - a) + a

                a = dx * (self.r[y    , x + 1] - self.r[y    , x]) + self.r[y    , x]
                b = dx * (self.r[y + 1, x + 1] - self.r[y + 1, x]) + self.r[y + 1, x]
                self.rtmp[j, i] = dy * (b - a) + a

        for i in np.arange(self.size, dtype = int):
            for j in np.arange(self.size, dtype = int):
                self.r[j, i] = self.rtmp[j, i]

        for i in np.arange(self.size, dtype = int)[1 : -1]:
            self.utmp[ 0, i] = -self.utmp[ 1, i]
            self.vtmp[ 0, i] = -self.vtmp[ 1, i]
            self.utmp[-1, i] = -self.utmp[-2, i]
            self.vtmp[-1, i] = -self.vtmp[-2, i]

        for j in np.arange(self.size, dtype = int):
            self.utmp[j,  0] = -self.utmp[j,  1]
            self.vtmp[j,  0] = -self.vtmp[j,  1]
            self.utmp[j, -1] = -self.utmp[j, -2]
            self.vtmp[j, -1] = -self.vtmp[j, -2]
    
    
    def addForce(self):
        f = 10.
        
        for i in np.arange(self.size / 2 - self.forceRange, self.size / 2 + self.forceRange, 1, dtype = int):
            for j in np.arange(self.size / 2 - self.forceRange * 2, self.size / 2, 1, dtype = int):
                self.vtmp[j, i] += self.step * f
    
    
    def solvePoisson(self):
        for t in np.arange(self.iteration):
            
            for i in np.arange(self.size, dtype = int)[1 : -1]:
                for j in np.arange(self.size, dtype = int)[1 : -1]:
                    b = (self.utmp[j    , i + 1] - self.utmp[j    , i - 1]) / (2 * self.scale) +\
                        (self.vtmp[j + 1, i    ] - self.vtmp[j - 1, i    ]) / (2 * self.scale)
                    
                    self.ptmp[j, i] = (self.p[j, i + 1] + self.p[j, i - 1] + self.p[j + 1, i] + self.p[j - 1, i] - (self.scale ** 2) * b) / 4.
            
            for i in np.arange(self.size, dtype = int)[1 : -1]:
                self.ptmp[ 0, i] = self.ptmp[ 1, i]
                self.ptmp[-1, i] = self.ptmp[-2, i]
            
            for j in np.arange(self.size, dtype = int):
                self.ptmp[j,  0] = self.ptmp[j,  1]
                self.ptmp[j, -1] = self.ptmp[j, -2]
            
            for i in np.arange(self.size, dtype = int):
                for j in np.arange(self.size, dtype = int):
                    self.p[j, i] = self.ptmp[j, i]
    
    
    def subtractPressureGradient(self):
        for i in np.arange(self.size, dtype = int)[1 : -1]:
            for j in np.arange(self.size, dtype = int)[1 : -1]:
                self.u[j, i] = self.utmp[j, i] - (self.ptmp[j    , i + 1] - self.ptmp[j    , i - 1]) / (2. * self.scale)
                self.v[j, i] = self.vtmp[j, i] - (self.ptmp[j + 1, i    ] - self.ptmp[j - 1, i    ]) / (2. * self.scale)
        
        for i in np.arange(self.size, dtype = int)[1 : -1]:
            self.u[ 0, i] = -self.u[ 1, i]
            self.v[ 0, i] = -self.v[ 1, i]
            self.u[-1, i] = -self.u[-2, i]
            self.v[-1, i] = -self.v[-2, i]
        
        for j in np.arange(self.size, dtype = int):
            self.u[j,  0] = -self.u[j,  1]
            self.v[j,  0] = -self.v[j,  1]
            self.u[j, -1] = -self.u[j, -2]
            self.v[j, -1] = -self.v[j, -2]
    
    
    def render(self, fileName):
        fig, sub = plt.subplots()
        
        plot = sub.imshow(self.r, cmap = "binary", vmin = 0., vmax = 1.)
        cbar = fig.colorbar(plot)
        
        sub.set_ylim(0, self.size)
        
        plt.savefig("./HW5/FLUID/%s.png" % fileName, dpi = 500, bbox_inches = "tight")
        plt.close()
    
    def run(self):
        for i in np.arange(self.size, dtype = int):
            for j in np.arange(self.size, dtype = int):
                self.p[j, i] = pow(10., 5)
                self.u[j, i] = 0.
                self.v[j, i] = 0.
                
                if np.sqrt((self.size / 2 - i) ** 2 + ((self.size / 2 - self.forceRange) - j) ** 2) < self.forceRange:
                    self.r[j, i] = 1.
                
                else:
                    self.r[j, i] = 0.
        
        a   = 0
        num = 1
        
        for t in tqdm(np.arange(self.step, self.end, self.step), total = len(np.arange(self.step, self.end, self.step))):
            self.advect()
            self.addForce()
            self.solvePoisson()
            self.subtractPressureGradient()
            
            if a%5 == 0:
                self.render("%s" % num)
                num += 1
            
            a += 1

In [58]:
size      = 250
forceRange = 20
iteration = 20
scale      = 0.1
step      = 0.01
end        = 10

In [60]:
sim = Fluid(size, step, scale, forceRange, iteration, end)
sim.run()

 51%|█████     | 510/999 [1:09:52<1:07:00,  8.22s/it]


KeyboardInterrupt: 